In [31]:
from tensorflow import keras

In [32]:
IMG_SIZE = (224, 224)

train_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='training')

val_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='validation')

Found 25000 files belonging to 2 classes.
Using 17500 files for training.
Found 25000 files belonging to 2 classes.
Using 7500 files for validation.


Per prima cosa, proviamo ad utilizzare la rete in **transfer learning**. Per farlo:

* selezioniamo una rete (in questo caso, `MobileNetV3Small`) dal package `applications` di Keras;
* specifichiamo il parametro `weights` ad `imagenet`, in modo da caricare i pesi della rete già addestrata su ImageNet;
* rimuoviamo il layer di classificazione di ImageNet impostando `include_top` a `False`;
* impostiamo l'attributo `trainable` a `False` per evitare di modificare i pesi della rete;
* costruiamo un nuovo modello.

In [33]:
base_model = keras.applications.MobileNetV3Small(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False
)

base_model.trainable = False

# sequential
model = keras.Sequential()
model.add(keras.layers.Lambda(keras.applications.mobilenet_v3.preprocess_input, input_shape=(224, 224, 3)))
model.add(base_model)
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1))

model.compile(
    optimizer=keras.optimizers.SGD(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()]
)

model.fit(train_ds, epochs=20, validation_data=val_ds)

Epoch 1/20
1094/1094 [==============================] - 150s 135ms/step - loss: 7.4448 - binary_accuracy: 0.5024 - val_loss: 7.6846 - val_binary_accuracy: 0.4975
Epoch 2/20
1094/1094 [==============================] - ETA: 0s - loss: 7.6225 - binary_accuracy: 0.5016

KeyboardInterrupt: 

Proviamo adesso ad effettuare il **fine tuning**. Per farlo, impostiamo `trainable` a `True`, e compiliamo il modello usando un learning rate molto basso (in questo caso, 0.00001).

In [ ]:
base_model.trainable = True

model.compile(
    optimizer=keras.optimizers.SGD(1e-5),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()],
)

model.fit(train_ds, epochs=10, validation_data=val_ds)